In [144]:
import torch
import einops

In [145]:
torch.manual_seed(0)

In [146]:
counts = torch.randint(0,4, (3, 5))

In [147]:
counts

tensor([[0, 3, 1, 0, 3],
        [3, 3, 3, 1, 3],
        [1, 2, 0, 3, 2]])

In [148]:
values, indices = torch.min(counts, dim=-1, keepdim=True)

In [149]:
indices.shape

torch.Size([3, 1])

In [150]:
values, indices

(tensor([[0],
         [1],
         [0]]),
 tensor([[0],
         [3],
         [2]]))

In [151]:
batch_size = 2

In [152]:
indices.unsqueeze(0).expand(batch_size, -1, -1)

tensor([[[0],
         [3],
         [2]],

        [[0],
         [3],
         [2]]])

In [153]:
sorts, sorti = torch.sort(counts)

In [154]:
sorts, sorti

(tensor([[0, 0, 1, 3, 3],
         [1, 3, 3, 3, 3],
         [0, 1, 2, 2, 3]]),
 tensor([[0, 3, 2, 1, 4],
         [3, 0, 1, 2, 4],
         [2, 0, 1, 4, 3]]))

In [155]:
sorti[:, :batch_size].T.reshape(batch_size, -1, 1)

tensor([[[0],
         [3],
         [2]],

        [[3],
         [0],
         [0]]])

In [156]:
# what if batch_size greater than the number of memories?
nodes, memes = sorti.shape

In [157]:
batch_size_xl = 23

In [158]:
full_expands = (batch_size_xl // memes) + 1

In [159]:
partial_expands = batch_size_xl % memes

In [160]:
partial_expands

3

In [161]:
full_expands

5

In [162]:
sorti

tensor([[0, 3, 2, 1, 4],
        [3, 0, 1, 2, 4],
        [2, 0, 1, 4, 3]])

In [163]:
sorti_expanded = sorti.unsqueeze(1).expand(-1, full_expands, -1).reshape(nodes, -1)

In [164]:
sorti_expanded

tensor([[0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1,
         4],
        [3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2,
         4],
        [2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4,
         3]])

In [165]:
sorti_selection = sorti_expanded[:, :(full_expands-1)*memes + partial_expands]

In [166]:
sorti_selection, indices

(tensor([[0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2, 1, 4, 0, 3, 2],
         [3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1, 2, 4, 3, 0, 1],
         [2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1, 4, 3, 2, 0, 1]]),
 tensor([[0],
         [3],
         [2]]))

In [167]:
sorti_selection.T.reshape(batch_size_xl, -1, 1).shape

torch.Size([23, 3, 1])

In [168]:
def batch_min(tensor, batch_size: int):
    sorts, sorti = torch.sort(tensor)
    nodes, memes = sorti.shape
    full_expands = (batch_size // memes) + 1
    partial_expands = batch_size % memes
    sorti_expanded = sorti.unsqueeze(1).expand(-1, full_expands, -1).reshape(nodes, -1)
    sorti_selection = sorti_expanded[:, :(full_expands-1)*memes + partial_expands]
    return sorti_selection, sorti_selection.T.reshape(batch_size, -1, 1)

In [ ]:
def maxi(x):
  max_indices = torch.argmax(x, dim=-1, keepdim=True)
  blank = torch.zeros_like(x)
  return torch.scatter(blank, -1, max_indices, torch.gather(x, -1, max_indices))


def argmaxi(x, eps=1e-8):
  maxied = torch.abs(maxi(x))
  factors, indices = torch.max(maxied, dim=-1)
  factors = factors - eps
  return maxied / factors.unsqueeze(-1)


def growth_argmaxi(x, counts, eps=1e-8, threshold=0.9):
  batch_size, nodes, mems = x.shape
  normal_path = argmaxi(x, eps)
  indices, indices_sg = batch_min(counts, batch_size)
  # indices_sg = indices.unsqueeze(0).expand(batch_size,-1,-1)
  growth_path = torch.zeros_like(normal_path)
  # values_of_interest = torch.gather(x, -1, indices_sg)
  # taking a closer look, it doesn't seem like the exact values from
  # the original should matter, we were just using it to divide
  # by itself (minus an eps) in the argmaxi to turn them into 1s,
  # to just setting it to 1 should be fine?
  values_of_interest = 1
  growth_path = torch.scatter(growth_path, -1, indices_sg, values_of_interest)
  trigger_growth = torch.sum(x > threshold, dim=-1, keepdim=True) <= 0
  grown = torch.where(trigger_growth, growth_path, normal_path)

  #count_needs_update = einops.reduce(trigger_growth, 'batch nodes flag -> nodes', 'sum')
  count_values_of_interest = torch.gather(counts, -1, indices).T.reshape(batch_size, -1, 1)
  res = torch.where(trigger_growth, 0, count_values_of_interest)
  batched_counts = counts.unsqueeze(0).expand(batch_size, -1, -1)
  updated_batch_counts = torch.scatter(batched_counts, -1, indices_sg, res)
  updated_counts = einops.reduce(updated_batch_counts, 'batch nodes mems -> nodes mems', 'min')
  updated_counts = updated_counts + einops.reduce(grown, 'batch nodes mems -> nodes mems', 'sum')

  return grown, updated_counts.detach()

In [ ]:
test = torch.tensor([
    [
        [0.2, 0.3, 0.5],
        [0.99, 0.01, 0.0],
        [0.91, 0.93, 0.92],
        [0.3, 0.3, 0.4],
    ],
    [
        [0.1, 0.2, 0.7],
        [0.1, 0.1, 0.8],
        [0.1, 1.0, 0.8],
        [0.1, 0.1, 0.8],
    ],
])
counts = torch.tensor([
        [12, 11, 13],
        [10, 14, 30],
        [15, 16, 9],
        [3, 2, 1],
])

In [305]:
grown, updated = growth_argmaxi(test, counts)

normal_path
tensor([[[0., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.]]])
growth_path
tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.]]])
grown
tensor([[[0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]])
tensor([[[12,  0, 13],
         [10, 14, 30],
         [15, 16,  9],
         [ 3,  2,  0]],

        [[ 0, 11, 13],
         [10,  0, 30],
         [15, 16,  9],
         [ 3,  0,  1]]])
